In [ ]:
!pip install pillow

In [ ]:
import os
from PIL import Image

# Read the image
I = Image.open('6.png')

# Get the size of the image
cols, rows = I.size
channels = len(I.getbands())

# Define the size of each patch
patchSize = 100

# Create a new folder to store the patches
folderName = 'patches'
if not os.path.exists(folderName):
    os.makedirs(folderName)

# Initialize a counter for naming patches
patchCount = 1

# Loop over the image in blocks of 20x20
for i in range(0, rows, patchSize):
    for j in range(0, cols, patchSize):
        # Extract the patch
        if (i+patchSize <= rows) and (j+patchSize <= cols):
            patch = I.crop((j, i, j+patchSize, i+patchSize))
            # Save the patch
            patch.save(f'{folderName}/patch_{patchCount}.png')
            patchCount += 1


In [ ]:
import os
import cv2

# Read the image
I = cv2.imread('6.png')

# Get the size of the image
rows, cols, channels = I.shape

# Define the size of each patch
patchSize = 100  # You can change this value to any other

# Create a new folder to store the patches
folderName = 'patches'
os.makedirs(folderName, exist_ok=True)

# Initialize a counter for naming patches
patchCount = 1

# Loop over the image in blocks of patchSize x patchSize
for i in range(0, rows, patchSize):
    for j in range(0, cols, patchSize):
        # Determine the patch dimensions, adjusting for the edge cases
        rowEnd = min(i+patchSize, rows)
        colEnd = min(j+patchSize, cols)

        # Extract the patch
        patch = I[i:rowEnd, j:colEnd, :]

        # Save the patch
        patchFileName = os.path.join(folderName, f'patch_{patchCount}.png')
        cv2.imwrite(patchFileName, patch)
        patchCount += 1


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
  layers.InputLayer(input_shape=(384, 384, 3)),
  layers.Conv2D(64, (1, 1), activation='relu', padding='same'),
  layers.MaxPooling2D(pool_size=(8, 8)),
  layers.Conv2D(64, (1, 1), activation='relu', padding='same'),
  layers.MaxPooling2D(pool_size=(8, 8)),
  layers.Conv2D(128, (1, 1), activation='relu', padding='same'),
  layers.Conv2D(64, (1, 1), activation='relu', padding='same'),
  layers.Dropout(0.5),
  layers.Conv2D(3, (1, 1), activation='relu', padding='same'),
  layers.GlobalAveragePooling2D(),
  layers.Dense(3)
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 384, 384, 64)      256       
                                                                 
 max_pooling2d (MaxPooling2  (None, 48, 48, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        4160      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 128)         8320      
                                                                 
 conv2d_3 (Conv2D)           (None, 6, 6, 64)          8

In [ ]:
import tensorflow as tf
from tensorflow.keras import callbacks


def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square((y_pred - y_true) / y_true))

optimizer = tf.optimizers.AdamW(learning_rate=2e-3, weight_decay=5e-5)

model.compile(optimizer= optimizer, loss= custom_loss)

class CyclicLR(callbacks.Callback):
    def _init_(self, max_lr, min_lr, half_cycle):
        self.max_lr = max_lr
        self.min_lr = min_lr
        self.half_cycle = half_cycle
        self.iteration = 0
        super(CyclicLR, self)._init_()

    def on_epoch_begin(self, epoch, logs=None):
        cycle = tf.floor(1 + self.iteration / (2 * self.half_cycle))
        x = tf.abs(self.iteration / self.half_cycle - 2 * cycle + 1)
        new_lr = self.min_lr + (self.max_lr - self.min_lr) * tf.maximum(0, (1 - x))
        self.iteration += 1
        self.model.optimizer.learning_rate.assign(new_lr)



In [ ]:

model.load_weights('/content/light_esti.h5')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
test_folder = '/content/patches'

# Get the list of image file names in the test folder
test_files = os.listdir(test_folder)

# Iterate over each image file, load it, preprocess it, and make predictions
predictions_list = []
for filename in test_files:
    img_path = os.path.join(test_folder, filename)
    img = image.load_img(img_path, target_size=(384, 384))  # Assuming the same target size as during training
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize pixel values
    prediction = model.predict(img_array)
    predictions_list.append(prediction)

# Convert predictions list to a numpy array
predictions_array = np.array(predictions_list)

# Print predictions
print("Predictions:", predictions_array)

1/1 [==============================] - 0s 28ms/step
Predictions: [[[0.32864934 0.8353279  0.7228495 ]]

 [[0.19633311 0.5102718  0.3611666 ]]

 [[0.26878834 0.7577553  0.6414897 ]]

 [[0.43813178 1.2524183  1.2156649 ]]

 [[0.18323395 0.4661094  0.31070158]]

 [[0.18535125 0.47330737 0.31920677]]

 [[0.40771568 1.1369725  1.0868139 ]]

 [[0.18627934 0.47602332 0.32235834]]

 [[0.20120794 0.5264852  0.3796039 ]]

 [[0.42861867 1.2372379  1.1956025 ]]

 [[0.3126375  0.88466233 0.7890241 ]]

 [[0.44773236 1.2988029  1.265953  ]]

 [[0.25989804 0.7283428  0.6080163 ]]

 [[0.38249272 1.0883961  1.0255675 ]]

 [[0.26754287 0.75380224 0.6369622 ]]

 [[0.34086278 0.69418645 0.5347926 ]]

 [[0.288665   0.792402   0.6811767 ]]

 [[0.21995993 0.59489    0.45636442]]

 [[0.45041135 1.311177   1.2794352 ]]

 [[0.37075368 0.791871   0.65468717]]

 [[0.28555366 0.80827326 0.69982857]]

 [[0.25735414 0.72063017 0.5991185 ]]

 [[0.33718595 0.9002368  0.80844516]]

 [[0.43721685 1.2662542  1.2285273 ]]


In [ ]:
from scipy.io import savemat
# Save predictions as a .mat file
savemat('/content/predictions.mat', {'predictions': predictions_array})